In [15]:
import pandas as pd; import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import warnings; import os 

import tensorflow as tf 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.models import Sequential 
from tensorflow.keras import optimizers
building_id = 1 

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

appliances_states_sel = ['Temp_mean_p0','sen_hou','temp_OA','Occ']
appliances_preds_sel = ['appliances_pow']

lighting_states_sel = ['Temp_mean_p0','sen_hou','temp_OA','hir_sol','Occ']
lighting_preds_sel = ['elec_lights']

min_max = {}

if building_id == 1: 
    data = '0_Data/Case_1/comb_data.csv'
    min_max["hvac_dem_tot"] = [0,6020] #1  #kW 
    min_max['appliances_pow'] = [0,42] #1 #kW
    min_max['elec_lights'] = [0,70]   #1 #kW
    min_max['Occ'] = [0,3300]
    scale_delta, scale_hvac = 200 , 200
elif building_id == 6:
    data = '0_Data/Case_6/comb_data.csv'
    min_max["hvac_dem_tot"] = [0,1620] #6  #kW 
    min_max['appliances_pow'] = [0,160] #6 #kW
    min_max['elec_lights'] = [0,190]   #6 #kW
    min_max['Occ'] = [0,250]
    scale_delta, scale_hvac = 600 , 200
      

min_max['u_heat_stp'] = [15,25] 
min_max['u_cool_stp'] = [20,30] 
min_max['delta_T'] = [0,1] 
min_max['Stp_n'] = [0,1] 
min_max['mode'] = [0,1] 

# lighting and appliances follow a random scale where the pred values are limited between 0-10
min_max['elec_equip'] = [0,35]    #kW

path = '0_Data/Case_1/' 
path_NN = path + '02_NN/'

In [16]:
# path_save = '2_EnergyPlus/6_Buckhead/'
path_save = '0_Data/Case_1/'
df = pd.read_csv(path_save+"df.csv")

keys = list(set(lighting_states_sel + lighting_preds_sel + appliances_states_sel + appliances_preds_sel))


In [17]:
df_n = df[:]

for key in keys:
     df_n[key] = (df_n[key] - min_max[key][0])/(min_max[key][1]-min_max[key][0])

df_n[keys].head()

KeyError: 'hir_sol'

In [ ]:
def get_light_nn_model(lr,state_size,pred_size):          
    model = Sequential()
    model.add(Dense(800,input_dim=state_size,activation="tanh",kernel_initializer = 'he_uniform'))                            
    model.add(Dense(1200, activation="relu", kernel_initializer='he_uniform'))          
    model.add(Dense(1200, activation="relu", kernel_initializer='he_uniform'))
    model.add(Dense(800, activation="relu", kernel_initializer='he_uniform'))
    model.add(Dense(pred_size, activation="relu", kernel_initializer='he_uniform'))
    model.summary()
    # loss = tf.keras.losses.Huber(delta=1)
    loss=tf.keras.losses.MeanAbsoluteError()

    # model.compile(loss=loss,optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-07))
    model.compile(loss=loss,optimizer = tf.keras.optimizers.Adam(learning_rate=lr))
    return model    

def get_appl_nn_model(lr,state_size,pred_size):          
    model = Sequential()
    model.add(Dense(800,input_dim=state_size,activation="tanh",kernel_initializer = 'he_uniform'))                            
    model.add(Dense(1200, activation="tanh", kernel_initializer='he_uniform'))          
    model.add(Dense(1200, activation="relu", kernel_initializer='he_uniform'))
    model.add(Dense(1000, activation="relu", kernel_initializer='he_uniform'))
    model.add(Dense(pred_size, activation="relu", kernel_initializer='he_uniform'))
    model.summary()
    # loss = tf.keras.losses.Huber(delta=0.1)
    loss=tf.keras.losses.MeanAbsoluteError()

    # model.compile(loss=loss,optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-07))

    model.compile(loss=loss,optimizer = tf.keras.optimizers.Adam(learning_rate=lr))


    return model   




In [7]:
view_no =10
start = 106149

df.iloc[start:start+view_no][keys]

,hir_sol,Occ,appliances_pow,sen_hou,temp_OA,elec_lights,Temp_mean_p0
106149,258.000000,0.0,31.620213,4.500000,-5.6,34.093888,18.999978
106150,258.333333,0.0,31.620213,4.666667,-5.6,34.093888,19.948571
106151,258.666667,0.0,31.620213,4.833333,-5.6,34.093888,19.999957
106152,259.000000,0.0,31.620213,5.000000,-5.6,34.093888,19.999943
106153,259.333333,0.0,31.620213,5.166667,-5.5,34.093888,19.999948
106154,259.666667,0.0,31.620213,5.333333,-5.4,34.093888,19.179263
106155,260.000000,0.0,31.620213,5.500000,-5.3,34.093888,19.000225
106156,260.333333,0.0,31.620213,5.666667,-5.2,34.093888,19.000003
106157,260.666667,0.0,31.620213,5.833333,-5.1,34.093888,20.579809
106158,261.000000,0.0,31.620213,6.000000,-5.0,34.093888,20.973603


In [36]:
df_n.iloc[start:start+view_no][keys]

,temp_OA,hir_sol,Temp_mean_p0,appliances_pow,Occ,elec_lights,sen_hou
106149,0.240000,0.368571,-0.375496,0.208546,0.0,0.180366,0.187500
106150,0.240000,0.369048,-0.376541,0.208546,0.0,0.180366,0.194444
106151,0.240000,0.369524,-0.377583,0.208546,0.0,0.180366,0.201389
106152,0.240000,0.370000,-0.378618,0.208546,0.0,0.180366,0.208333
106153,0.241667,0.370476,-0.379321,0.208546,0.0,0.180366,0.215278
106154,0.243333,0.370952,-0.379654,0.208546,0.0,0.180366,0.222222
106155,0.245000,0.371429,-0.379752,0.208546,0.0,0.180366,0.229167
106156,0.246667,0.371905,-0.379743,0.208546,0.0,0.180366,0.236111
106157,0.248333,0.372381,-0.379690,0.208546,0.0,0.180366,0.243056
106158,0.250000,0.372857,-0.379609,0.208546,0.0,0.180366,0.250000


In [20]:
df_n[['appliances_pow','elec_lights']].max()

appliances_pow    0.998647
elec_lights       0.999922
dtype: float64

In [8]:
appliances_states_sel = ['Temp_mean_p0','sen_hou','temp_OA','Occ']
appliances_preds_sel = ['appliances_pow']

X_appliances = df_n[appliances_states_sel]
y_appliances = df_n[appliances_preds_sel]

In [18]:
appliances_epochs = 40
batch_size_appliances = 2000

lr_appliances = 0.00002

appliances_model  = get_appl_nn_model(lr_appliances,len(appliances_states_sel),1)

appliances_model.load_weights(path_NN+"appliances_model.h5")

appliances_history = appliances_model.fit(X_appliances,y_appliances, epochs=appliances_epochs, batch_size= batch_size_appliances,verbose=1)
appliances_model.save_weights(path_NN+"appliances_model.h5")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 800)               4000      
                                                                 
 dense_11 (Dense)            (None, 1200)              961200    
                                                                 
 dense_12 (Dense)            (None, 1200)              1441200   
                                                                 
 dense_13 (Dense)            (None, 1000)              1201000   
                                                                 
 dense_14 (Dense)            (None, 1)                 1001      
                                                                 
Total params: 3,608,401
Trainable params: 3,608,401
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
1420/1420 [======================

In [14]:
a = 1800 
b = a+10 

pd.concat([X_appliances[a:b],y_appliances[a:b]],axis=1)

,Temp_mean_p0,sen_hou,temp_OA,Occ,appliances_pow
1800,0.398925,0.513889,0.447778,0.060586,0.968029
1801,0.401945,0.520833,0.449167,0.060586,0.984436
1802,0.404541,0.527778,0.450556,0.060586,0.984436
1803,0.406684,0.534722,0.451944,0.060586,0.984436
1804,0.408771,0.541667,0.453333,0.060586,0.984436
1805,0.411406,0.548611,0.453333,0.075732,0.984436
1806,0.414186,0.555556,0.453333,0.075732,0.984436
1807,0.416850,0.562500,0.453333,0.075732,0.984436
1808,0.419489,0.569444,0.453333,0.075732,0.984436
1809,0.422211,0.576389,0.453333,0.075732,0.984436


In [12]:
X_light = df_n[lighting_states_sel]
y_light = df_n[lighting_preds_sel]

pd.concat([X_light[a:b],y_light[a:b]],axis=1)

,Temp_mean_p0,sen_hou,temp_OA,hir_sol,Occ,elec_lights
2000,0.037022,0.902778,0.343333,0.380476,0.00342,0.23513
2001,0.035359,0.909722,0.343333,0.383095,0.00342,0.23513
2002,0.033740,0.916667,0.343333,0.385714,0.00342,0.23513
2003,0.031741,0.923611,0.341667,0.385238,0.00000,0.23513
2004,0.029432,0.930556,0.340000,0.384762,0.00000,0.23513
2005,0.026998,0.937500,0.338333,0.384286,0.00000,0.23513
2006,0.024549,0.944444,0.336667,0.383810,0.00000,0.23513
2007,0.022108,0.951389,0.335000,0.383333,0.00000,0.23513
2008,0.019668,0.958333,0.333333,0.382857,0.00000,0.23513
2009,0.017490,0.965278,0.333333,0.381429,0.00000,0.23513


elec_lights    10.14416
dtype: float64

In [19]:
X_light = df_n[lighting_states_sel]
y_light = df_n[lighting_preds_sel]

lighting_epochs = 50
batch_size_lighting = 2000

lr_lighting = 0.000001

lighting_model  = get_light_nn_model(lr_appliances,len(lighting_states_sel),1)

lighting_model.load_weights(path_NN+"lighting_model.h5")

lighting_history = lighting_model.fit(X_light,y_light, epochs=lighting_epochs, batch_size= batch_size_lighting,verbose=1)
lighting_model.save_weights(path_NN+"lighting_model.h5")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 800)               4800      
                                                                 
 dense_16 (Dense)            (None, 1200)              961200    
                                                                 
 dense_17 (Dense)            (None, 1200)              1441200   
                                                                 
 dense_18 (Dense)            (None, 800)               960800    
                                                                 
 dense_19 (Dense)            (None, 1)                 801       
                                                                 
Total params: 3,368,801
Trainable params: 3,368,801
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1367/1367 [======================

In [43]:
X_light = df_n[lighting_states_sel]
y_light = df_n[lighting_preds_sel]

In [44]:
appliances_model.load_weights(path_NN+"appliances_model.h5")


X_appl_test = np.array([[0.45,0.17,0.528,3]])
appliances_model.predict(X_appl_test)

1/1 [==============================] - 0s 123ms/step


array([[0.]], dtype=float32)

In [45]:
start = 600
no = 5

lighting_model.predict(X_light[start:start+no])


1/1 [==============================] - 0s 48ms/step


array([[0.04229053],
       [0.0424193 ],
       [0.04237578],
       [0.04400878],
       [0.04543202]], dtype=float32)

In [46]:
y_light[start:start+no]

,elec_lights
600,0.055551
601,0.055551
602,0.055551
603,0.055551
604,0.055551


In [47]:
y_light.max()

elec_lights    0.999922
dtype: float64